In [ ]:
try:
    #import google.colab # type: ignore
    #from google.colab import output
    %pip install git+https://github.com/TKOHP/SAELens.git transformer-lens circuitsvis
except:
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [6]:
from sae_lens import CacheActivationsRunnerConfig,CacheActivationsRunner
from datasets import load_dataset
dataset=load_dataset("Duxiaoman-DI/FinCorpus")
cfg=CacheActivationsRunnerConfig(
    model_name="Duxiaoman-DI/XuanYuan-6B-Chat",
    model_class_name="LlamaForCausalLM",
    hook_name="blocks.0.hook_mlp_out",
    context_size=512,
    d_in=4096,
    training_tokens=1024*30_00,
    n_shuffles_final=100,# 存储完所有的buffer后，最终再进行n次shuffle（任意两个作shuffle）
    shuffle_every_n_buffers= 100,# 每存储n个buffer，作shuffle
    n_shuffles_with_last_section= 100,# 还不走
    n_shuffles_in_entire_dir= 100,# 还不知
    n_batches_in_buffer=8,# 和训练的一样
    store_batch_size_prompts=8,# 和训练的一样
    new_cached_activations_path="D:/project/dataset/activations/1",
    # ignore
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2"
)
a=CacheActivationsRunner(cfg,override_dataset=dataset).run()
print(a)


KeyboardInterrupt



In [1]:
import torch
import os
from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner
device = "cuda"
print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
total_training_steps = 30_00  # probably we should do more
batch_size = 1024
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    ######## 如果是存储好activations的话，下面配置忽略
    mmodel_name="Duxiaoman-DI/XuanYuan-6B-Chat",
    model_class_name="LlamaForCausalLM",
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",
    #######
    hook_name="blocks.0.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_layer=0,  # Only one layer in the model.
    d_in=4096,  # the width of the mlp output.
    is_dataset_tokenized=True,
    streaming=False,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=2**3,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # normalize_activations="none",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=8,  # controls how many activations we store / shuffle.
    # n_batches_in_buffer=8,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    # store_batch_size_prompts=32,
    store_batch_size_prompts=8,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project="sae_lens_tutorial",
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    # act_store_device="cpu",
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32",
    # store_activations
    use_cached_activations=True,
    cached_activations_path="D:/project/dataset/activations/1"
)
# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder = SAETrainingRunner(cfg).run()

D:\tools\anaconda3\envs\llm_test\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Using device: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type tinyllm to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


TypeError: HookedTransformer.__init__() missing 1 required positional argument: 'cfg'